In [339]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional: Improves WordNet performance

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/kaggle/input/social-media-sentiments-analysis-dataset/sentimentdataset.csv


In [340]:
import pandas as pd

train_df = pd.read_csv("/kaggle/input/social-media-sentiments-analysis-dataset/sentimentdataset.csv")

In [341]:
import nltk
import subprocess

nltk.download('wordnet', download_dir='/kaggle/working/')
nltk.download('omw-1.4', download_dir='/kaggle/working/')

command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
subprocess.run(command.split())

nltk.data.path.append('/kaggle/working/')

lemmatizer = WordNetLemmatizer()

train_df['Sentiment'] = train_df['Sentiment'].astype(str).str.strip().str.lower().apply(lemmatizer.lemmatize)
print(train_df['Sentiment'].value_counts())

[nltk_data] Downloading package wordnet to /kaggle/working/...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /kaggle/working/...
[nltk_data]   Package omw-1.4 is already up-to-date!
Sentiment
positive                45
joy                     44
excitement              37
contentment             19
neutral                 18
gratitude               18
curiosity               16
serenity                15
happy                   14
nostalgia               11
despair                 11
grief                    9
awe                      9
sad                      9
hopeful                  9
loneliness               9
embarrassed              8
acceptance               8
confusion                8
euphoria                 7
elation                  7
enthusiasm               7
pride                    7
determination            7
regret                   6
frustration              6
ambivalence              6
melancholy               6
nu

In [342]:
unique_values = set()
unique_values.update(train_df['Sentiment'])
unique_values

{'acceptance',
 'accomplishment',
 'admiration',
 'adoration',
 'adrenaline',
 'adventure',
 'affection',
 'amazement',
 'ambivalence',
 'amusement',
 'anger',
 'anticipation',
 'anxiety',
 'appreciation',
 'apprehensive',
 'arousal',
 'artisticburst',
 'awe',
 'bad',
 'betrayal',
 'bitter',
 'bitterness',
 'bittersweet',
 'blessed',
 'boredom',
 'breakthrough',
 'calmness',
 'captivation',
 'celebration',
 'celestial wonder',
 'challenge',
 'charm',
 'colorful',
 'compassion',
 'compassionate',
 'confidence',
 'confident',
 'confusion',
 'connection',
 'contemplation',
 'contentment',
 'coziness',
 'creative inspiration',
 'creativity',
 'culinary adventure',
 'culinaryodyssey',
 'curiosity',
 'darkness',
 'dazzle',
 'desolation',
 'despair',
 'desperation',
 'determination',
 'devastated',
 'disappointed',
 'disappointment',
 'disgust',
 'dismissive',
 'dreamchaser',
 'ecstasy',
 'elation',
 'elegance',
 'embarrassed',
 'emotion',
 'emotionalstorm',
 'empathetic',
 'empowerment',
 'e

In [343]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

nlp = spacy.load("en_core_web_md")

ref_words = {
    "Positive": "positive",
    "Negative": "negative",
    "Neutral": "neutral"
}

ref_vectors = {category: nlp(word).vector for category, word in ref_words.items()}

def assign_sentiment_category(sentiment):
    word_vector = nlp(sentiment).vector.reshape(1, -1)
    
    similarities = {}
    for category, ref_vec in ref_vectors.items():
        ref_vec = ref_vec.reshape(1, -1)
        sim = cosine_similarity(word_vector, ref_vec)[0][0]
        similarities[category] = sim

    return max(similarities, key=similarities.get)

train_df['Sentiment'] = train_df['Sentiment'].apply(assign_sentiment_category)

print(train_df['Sentiment'].value_counts())

Sentiment
Positive    364
Negative    290
Neutral      78
Name: count, dtype: int64


In [344]:
!pip install nltk  

import nltk
nltk.download('stopwords')  
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
import re
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F700-\U0001F77F" 
                           u"\U0001F780-\U0001F7FF"  
                           u"\U0001F800-\U0001F8FF" 
                           u"\U0001F900-\U0001F9FF" 
                           u"\U0001FA00-\U0001FA6F" 
                           u"\U0001FA70-\U0001FAFF" 
                           u"\U00002702-\U000027B0" 
                           u"\U000024C2-\U0001F251" 
                           "]+", flags=re.UNICODE)

def split_and_remove_stopwords(text):
    text = emoji_pattern.sub(r'', text)
    tokens = text.split()

    filtered_tokens = [
        stemmer.stem(lemmatizer.lemmatize(word.lower().rstrip(string.punctuation)))
        for word in tokens if stemmer.stem(word.lower().rstrip(string.punctuation)) not in stop_words
    ]

    return filtered_tokens

train_df['Text'] = train_df['Text'].apply(split_and_remove_stopwords)

print(train_df['Text'])

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
0                             [enjoy, beauti, day, park]
1                      [traffic, wa, terribl, thi, morn]
2                                [finish, amaz, workout]
3                       [excit, upcom, weekend, getaway]
4                     [tri, new, recip, dinner, tonight]
5                      [feel, grate, littl, thing, life]
6          [raini, day, call, cozi, blanket, hot, cocoa]
7                        [new, movi, releas, must-watch]
8                        [polit, discuss, heat, timelin]
9                       [miss, summer, vibe, beach, day]
10                     [publish, new, blog, post, check]
11       

In [345]:
from gensim.models import Word2Vec
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

# Train Word2Vec using skip-gram (sg=1) with 100 epochs
sentences = train_df['Text'].tolist()
w2v_model = Word2Vec(sentences, vector_size=300, window=7, min_count=2, workers=4, sg=1, negative=10, epochs=100)
w2v_model.save("word2vec.model")

# Function to generate a 1200-dimension sentence vector (avg, max, min, and std pooling)
def get_complex_sentence_vector(sentence, model, vector_size=300):
    vectors = [model.wv[word] for word in sentence if word in model.wv]
    if vectors:
        vectors = np.array(vectors)
        avg_vec = np.mean(vectors, axis=0)
        max_vec = np.max(vectors, axis=0)
        min_vec = np.min(vectors, axis=0)
        std_vec = np.std(vectors, axis=0)
        return np.concatenate([avg_vec, max_vec, min_vec, std_vec])
    else:
        return np.zeros(vector_size * 4)

train_df['vector'] = train_df['Text'].apply(lambda x: get_complex_sentence_vector(x, w2v_model))

label_encoder = LabelEncoder()
train_df['Sentiment'] = label_encoder.fit_transform(train_df['Sentiment'])
X = np.vstack(train_df['vector'].values)
y = train_df['Sentiment'].values.astype(np.int32)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = Sequential([
        Dense(2048, input_shape=(1200,)),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(1024),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(512),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.4),
        
        Dense(128),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.4),
        
        Dense(3, activation='softmax')
    ])
    
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)
    
    model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[lr_scheduler], verbose=1)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    accuracies.append(accuracy)
    
print(f"Mean Accuracy: {np.mean(accuracies):.4f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 100ms/step - accuracy: 0.3405 - loss: 1.7073 - val_accuracy: 0.4898 - val_loss: 1.0740 - learning_rate: 0.0010
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.4884 - loss: 1.3496 - val_accuracy: 0.5238 - val_loss: 1.0353 - learning_rate: 0.0010
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.5637 - loss: 1.0553 - val_accuracy: 0.5782 - val_loss: 0.9901 - learning_rate: 0.0010
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.6145 - loss: 1.0327 - val_accuracy: 0.5442 - val_loss: 1.0844 - learning_rate: 0.0010
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.6346 - loss: 0.9223 - val_accuracy: 0.5374 - val_loss: 1.2529 - learning_rate: 0.0010
Epoch 6/100
 9/10 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.6236 - loss: 0.9369
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.6253 - loss: 0.9331 - v